In [62]:
import os
import numpy as np
import cv2

folder_path = "C:\\Users\\nehab\\OneDrive\\Desktop\\test"

data = []
labels = []
#print (os.listdir(folder_path))
for folder_name in os.listdir(folder_path):
    #print (folder_name)
    folder_path_class = os.path.join(folder_path, folder_name)
    #print (folder_path_class)
    label = folder_name

    for image_path in os.listdir(folder_path_class):
        image_path_full = os.path.join(folder_path_class, image_path)
        image = cv2.imread(image_path_full)
        img=cv2.resize(image,(240,240))
        
        #gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        
        flat_image = img.flatten()

        data.append(flat_image)
        labels.append(label)



data1 = np.array(data,dtype=object)
labels1 = np.array(labels,dtype=object)
print (data1)
print (labels1)

[[16 27 57 ... 3 40 48]
 [31 67 46 ... 175 165 157]
 [178 154 124 ... 15 29 9]
 ...
 [95 161 186 ... 77 105 116]
 [124 157 130 ... 100 128 85]
 [147 159 155 ... 138 159 173]]
['artificial' 'artificial' 'artificial' 'artificial' 'artificial'
 'artificial' 'artificial' 'artificial' 'artificial' 'artificial'
 'artificial' 'artificial' 'artificial' 'artificial' 'artificial'
 'artificial' 'artificial' 'artificial' 'artificial' 'artificial'
 'artificial' 'artificial' 'artificial' 'artificial' 'artificial'
 'artificial' 'artificial' 'artificial' 'artificial' 'artificial'
 'artificial' 'artificial' 'artificial' 'artificial' 'artificial'
 'artificial' 'artificial' 'artificial' 'artificial' 'artificial'
 'artificial' 'artificial' 'artificial' 'artificial' 'artificial'
 'artificial' 'artificial' 'artificial' 'artificial' 'artificial'
 'artificial' 'artificial' 'artificial' 'artificial' 'artificial'
 'artificial' 'artificial' 'artificial' 'artificial' 'artificial'
 'artificial' 'artificial' 'artif

In [63]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data1, 
                                                labels1,random_state=0, 
                                                test_size=0.2)


print (X_test)
print (y_test)

[[252 253 253 ... 253 254 253]
 [253 252 252 ... 252 252 252]
 [224 235 239 ... 94 97 75]
 ...
 [180 154 103 ... 142 107 33]
 [252 253 252 ... 252 252 252]
 [16 10 14 ... 37 43 62]]
['artificial' 'artificial' 'human' 'human' 'human' 'artificial' 'human'
 'human' 'artificial' 'artificial' 'artificial' 'artificial' 'artificial'
 'human' 'artificial' 'human' 'human' 'artificial' 'human' 'artificial'
 'human' 'human' 'artificial' 'human' 'human' 'human' 'artificial' 'human'
 'human' 'human' 'artificial' 'artificial' 'artificial' 'human' 'human'
 'human' 'human' 'human' 'artificial' 'artificial' 'artificial' 'human'
 'artificial' 'human' 'human' 'artificial' 'artificial' 'human'
 'artificial' 'human' 'artificial' 'human' 'human' 'artificial'
 'artificial' 'human' 'artificial' 'artificial']


In [64]:
from sklearn.svm import SVC
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, plot_confusion_matrix
from sklearn.model_selection import KFold, cross_val_score
from matplotlib.colors import ListedColormap
from sklearn.svm import SVC
import seaborn as sns
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [30]:
def SVC_crossval(X_train,y_train,kernels,C_vals):
    kf = KFold(n_splits=3)
    best_acc=0
    for i in kernels:
        for j in C_vals:
            model=SVC(kernel=i,C=j)
            acc_folds=[]
            for (train_idx,val_idx) in kf.split(X=X_train,y=y_train):
                X_fold_train, X_fold_val = X_train[train_idx], X_train[val_idx]
                y_fold_train, y_fold_val = y_train[train_idx], y_train[val_idx]
                model.fit(X_fold_train,y_fold_train)
                preds=model.predict(X_fold_val)
                acc= sum([preds == y_fold_val])/len(y_fold_val)
                acc_folds.append(acc)
            mean_acc = np.mean(acc_folds)
            #print (mean_acc)
            if mean_acc > best_acc:
                best_acc = mean_acc
                best_c_val = j
                best_kernel =i
                #print (best_c_val,best_kernel)
    return(best_kernel,best_c_val,mean_acc)

In [65]:
def test_SVC_model(kernel,C_val,X_train,y_train,X_test):
    model=SVC(kernel=kernel,C=C_val)
    model.fit(X_train,y_train)
    preds=model.predict(X_test)
    return (preds)

In [66]:
y_preds = test_SVC_model('rbf',10,X_train,y_train,X_test)
acc = accuracy_score(y_test,y_preds)
print("Your model's accuracy on the test set is {:.3f}".format(acc))

Your model's accuracy on the test set is 0.897


In [67]:
def KNN (X_train,y_train,X_test) :
    knn_model = KNeighborsClassifier(n_neighbors=1)
    knn_model.fit(X_train,y_train)
    preds=knn_model.predict(X_test)
    return (preds)

In [68]:
y_preds_1=KNN(X_train,y_train,X_test)
acc=accuracy_score(y_test,y_preds_1)
print ("Your model's accuracy on the test set is {:.3f}".format(acc))

Your model's accuracy on the test set is 0.741


In [69]:
def tree_model(X_train,X_test,y_train,y_test):
    print (X_train)
    dtree_model = DecisionTreeClassifier(random_state=0)
    dtree_model.fit(X_train,y_train)
    pred=dtree_model.predict(X_test)
    test_acc=sum(pred == y_test)/len(y_test)
    return(test_acc)

In [70]:
tree_acc = tree_model(X_train,X_test,y_train,y_test)
print("Your tree's test set accuracy is {:.3f}".format(tree_acc))

[[88 107 85 ... 164 164 163]
 [255 255 255 ... 255 255 255]
 [136 166 166 ... 197 202 203]
 ...
 [35 45 59 ... 68 76 97]
 [203 243 248 ... 230 251 249]
 [164 185 187 ... 25 33 23]]
Your tree's test set accuracy is 0.810


In [71]:
def rf(X_train,X_test,y_train,y_test):    
    rf = RandomForestClassifier(min_samples_leaf=10,max_features=1,random_state=0)
    rf.fit(X_train,y_train)
    pred_rf=rf.predict(X_test)
    test_acc=sum(pred_rf == y_test)/len(y_test)
    return(test_acc)

In [72]:
rf_acc = rf(X_train,X_test,y_train,y_test)
print("Your tree's test set accuracy is {:.3f}".format(rf_acc))

Your tree's test set accuracy is 0.897
